# Лабораторная работа №1
## Градиентный спуск
Трутнев Алексей Игоревич вариант №5

In [23]:
import time
from dataclasses import dataclass

import numpy as np
from numpy.typing import NDArray
from typing import Callable

import cvxpy as cp

Задача оптимизации

$\min\{ (x-\mu_0)^\top A (x-\mu_0) : ||x||_2^2 \leq 1 \}$, где
* $x \in \mathbb{R}^n$, $A$ - симметричная, положительно определенная матрица
* $\mu_0 = (1,1,\dots,1)^\top \in \mathbb{R}^n$

In [2]:
def function(x: NDArray, A: NDArray, mu: NDArray) -> float:
    return (x - mu).T @ A @ (x - mu)


def limitation(x: NDArray) -> float:
    return np.sum(np.power(x, 2))

### Constraint gradient
$\sum_{i=1}^{n} x_i^2 \leq 1$, где $x = (x_1, x_2, \dots, x_n)^\top \in \mathbb{R}^n$

$\nabla = (2x_1, 2x_2, \dots, 2x_n)^\top \in \mathbb{R}^n$


### Objective gradient

$\nabla x^\top A x = (A+A^\top)x$. При симметричной матрице $A$ градиент имеет вид $\nabla x^\top A x = 2Ax$

In [3]:
def grad_function(x: NDArray, A: NDArray, mu: NDArray) -> NDArray:
    return 2 * A @ (x - mu)


def grad_limitation(x: NDArray) -> NDArray:
    return 2 * x

In [4]:
def check_optimality(x: NDArray, A: NDArray, mu: NDArray) -> bool:
    return grad_limitation(x) @ grad_function(x, A, mu) >= 0

1. Исследование на выпуклость

2. Для каждого значения $n \in \{10, 20,\dots, 100\}$ сгенерируйте $N = 100$ тестовых примеров. В каждом случае найдите глобальный минимум, $x^* \in \mathbb{R}^n$, с помощью CVX. Проверьте, что в точке минимума выполняется условие оптимальности (т.е. вектора градиента к ограничению и антиградиента к целевой функции сонаправлены).


In [50]:
def check_symmetric(A: NDArray) -> bool:
    return (A==A.T).all()


def check_positive_definition(A: NDArray) -> bool:
    n = A.shape[0]
    test_x = np.random.random(n)
    return (test_x.T @ A @ test_x) > 0


def get_symmetric_matrix(n: int) -> NDArray:
    A = np.random.rand(n, n)
    matrix = A @ A.T
    assert check_symmetric(matrix) and check_positive_definition(matrix)
    return matrix

In [51]:
dimensionality_range = range(10, 100, 10) # The n = 100 is not supported by solver
num_experiments = 1 # N =100


for n in dimensionality_range:
    mu = np.ones(n)

    for _ in range(num_experiments):
        matrix_A = get_symmetric_matrix(n)

        x = cp.Variable(n)
        optimization_func = cp.quad_form(x=(x - mu), P=matrix_A)
        objective = cp.Minimize(optimization_func)

        constraints = [
            cp.sum_squares(x) <= 1
        ]

        problem = cp.Problem(objective, constraints)
        problem.solve(solver=cp.ECOS, abstol=1e-6)

        x_opt = x.value

        is_optimal = check_optimality(x_opt, matrix_A, mu)
        if not is_optimal:
            print(f"Optimality condition not satisfied for n={n}.")

Optimality condition not satisfied for n=10.
Optimality condition not satisfied for n=20.
Optimality condition not satisfied for n=30.
Optimality condition not satisfied for n=40.
Optimality condition not satisfied for n=50.
Optimality condition not satisfied for n=60.
Optimality condition not satisfied for n=70.


/Users/alexgiving/Documents/PythonProjects/optimization/venv/lib/python3.11/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Optimality condition not satisfied for n=80.
Optimality condition not satisfied for n=90.


3. Для каждого значения $n \in \{10, 20,\dots, 100\}$ и для каждого тестового примера сгенерируйте 100 начальных точек. В зависимости от варианта реализуйте следующие методы решения задачи (1) для заданной точности $\epsilon = 0.011$.

### Gradient Descent

Given a starting point $x \in$ dom $f$.

While stopping criterion is satisfied:
1. Determine a gradient $\Delta x$.
2. *Line search*. Choose a step size $t > 0$.
3. Update. $x = x - t\Delta x$.

The stopping criterion is often of the form $||\nabla f(x)||_{2} \leq \eta$, $\eta > 0$

### Exact Line Search

One line search method sometimes used in practice is exact line search, in which $t$ is chosen to minimize $f$ along the ray $\{ x - t\Delta x | t \geq 0 \}$

$$t = \argmin_{s \geq 0}f(x - s \Delta x)$$

An exact line search is used when the cost of the minimization problem with one variable is low compared to the cost of computing the search direction itself. In some special cases the minimizer along the ray can be found analytically, and in others it can be computed efficiently.

Usually, it is not possible to do this minimization exactly.
Approximations to exact line search are often not much more efficient than backtracking, and it’s not worth it.

In [34]:
def print_iteration(iteration: int, y: int, x: NDArray) -> None:
    print(f'Iteration {iteration}, y: {y}, x: {x}')


def get_exact_line_alpha(func: Callable,
                         x: NDArray, A: NDArray, mu: NDArray, delta_x: NDArray,
                         max_s: float = 1., step_s: float = 0.001) -> float:
    s_values = np.arange(0, max_s, step_s)
    func_results = [func(x-s*delta_x, A, mu) for s in s_values]
    return s_values[np.argmin(func_results)]


@dataclass
class GradDescentResult:
    x: NDArray
    y: float
    time: float
    iterations: int


def grad_desc_exact_line(*,
                    n: int,
                    A: NDArray,
                    func: Callable,
                    grad_func: Callable,
                    epsilon: float = 1.1e-2,
                    max_iters: int = 1000,
                    max_s: float = 1.,
                    step_s: float = 0.001,
                    verbose: bool = False) -> GradDescentResult:
    """
    gradient descent with exact line minimization
    """
    start_time = time.time()

    x = np.random.random(n)
    mu = np.ones(n)

    if verbose:
        print_iteration(0, func(x, A, mu), x)

    for iteration in range(1, max_iters+1):

        delta_x = grad_func(x, A, mu)
        exact_alpha = get_exact_line_alpha(func, x, A, mu, delta_x, max_s, step_s)

        x -= exact_alpha * delta_x

        if verbose:
            print_iteration(iteration, func(x, A, mu), x)

        if np.linalg.norm(delta_x) < epsilon:
            if verbose:
                print('STOP Condition')
            break
    end_time = time.time()

    result = GradDescentResult(x, func(x, A, mu), end_time-start_time, iteration)
    if verbose:
        print(f'Optimal y: {result.y}, obtained in {result.x}')
    return result

In [35]:
n_values = range(10, 21, 10)
N = 3

for n in n_values:
    time_stamps = []
    iteration_stamps = []

    for experiment in range(N):
        A = get_symmetric_matrix(n)
        result = grad_desc_exact_line(n=n, A=A, func=function, grad_func=grad_function, verbose=False)

        time_stamps.append(result.time)
        iteration_stamps.append(result.iterations)

    print(f'[n={n}]: Avg time {np.mean(time_stamps):.6f} Avg iter {np.mean(iteration_stamps):.2f} ')

[n=10]: Avg time 0.508602 Avg iter 172.33 
[n=20]: Avg time 2.211027 Avg iter 728.33 


4. Объясните принцип работы метода, опишите его преимущества и недостатки.

5. В качестве результата работы метода:

* Для каждого значения $n \in \{10,20,\dots,100\}$ подсчитайте среднее время работы метода и среднее число итераций (усреднение проводится по всем начальным точкам и по всем тестовым примерам)

* Для одного тестового примера при $n = 10$ и нескольких различных начальных точек постройте зависимость тонности от числа итераций. Зависит ли скорость сходимости метода от отношения максимального и минимального собственных чисел матрицы $A$?

In [29]:
n = 10
N = 3

mu = np.ones(n)
time_stamps = []

for experiment in range(N):
    A = get_symmetric_matrix(n)

    start_time = time.time()
    x = grad_desc_exact_line(n=n, A=A, func=function, grad_func=grad_function)
    end_time = time.time()
    time_stamps.append(end_time - start_time)

    # print(f'Optimal y: {function(x, A, mu)}, obtained in {x}')
print(f'Average time for dimension n = {n} is {np.mean(time_stamps):.6f}')

Average time for dimension n = 10 is 1.617637
